## Main file for cleaning df4
### output file: clean_df4.csv

In [1]:
import ast
import pandas as pd
import json

records = []
with open("steam_games.json", "rt", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = ast.literal_eval(line)
        records.append(obj)

df4 = pd.DataFrame(records)

print(df4.shape)
print(df4.head())

(32135, 16)
          publisher                                             genres  \
0         Kotoshiro      [Action, Casual, Indie, Simulation, Strategy]   
1  Making Fun, Inc.               [Free to Play, Indie, RPG, Strategy]   
2      Poolians.com  [Casual, Free to Play, Indie, Simulation, Sports]   
3              彼岸领域                        [Action, Adventure, Casual]   
4               NaN                                                NaN   

                  app_name                    title  \
0      Lost Summoner Kitty      Lost Summoner Kitty   
1                Ironbound                Ironbound   
2  Real Pool 3D - Poolians  Real Pool 3D - Poolians   
3                  弹炸人2222                  弹炸人2222   
4            Log Challenge                      NaN   

                                                 url release_date  \
0  http://store.steampowered.com/app/761140/Lost_...   2018-01-04   
1  http://store.steampowered.com/app/643980/Ironb...   2018-01-04   
2  ht

In [2]:
df4_test = df4.copy()

In [3]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [ ]:
# only keep rows which have at least 2 out 3 features: genres, tags, specs

def drop_rows_with_2_missing_features(df, features):
    return df.dropna(thresh=len(features)-1, subset=features)

features = ['genres', 'tags', 'specs']
df4_test = drop_rows_with_2_missing_features(df4_test, features)
df4_test.shape

(31618, 16)

In [5]:
df4_test = df4_test.dropna(subset=['tags', 'specs'])

In [6]:
features = ['genres', 'tags', 'specs']
for feature in features:
    df4_test[feature] = df4_test[feature].apply(lambda x: x[:len(x)] if isinstance(x, list) else None)

In [ ]:
# Merge title/app_name

for col in ['title', 'app_name']:
    df4_test[col] = df4_test[col].astype('string').str.strip().replace({'': pd.NA})

df4_test['title'] = df4_test['title'].fillna(df4_test.get('app_name'))
df4_test = df4_test[df4_test['title'].notna()].reset_index(drop=True)
df4_test = df4_test.drop(columns=['app_name'])

In [10]:
df4_test.drop(columns=['discount_price', 'metascore'], inplace=True)

In [11]:
df4_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31311 entries, 0 to 31310
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     23702 non-null  object
 1   genres        28543 non-null  object
 2   title         31311 non-null  string
 3   url           31311 non-null  object
 4   release_date  29246 non-null  object
 5   tags          31311 non-null  object
 6   reviews_url   31310 non-null  object
 7   specs         31311 non-null  object
 8   price         29977 non-null  object
 9   early_access  31311 non-null  bool  
 10  id            31310 non-null  object
 11  developer     28417 non-null  object
 12  sentiment     24371 non-null  object
dtypes: bool(1), object(11), string(1)
memory usage: 2.9+ MB


In [ ]:
df4_test['release_date'] = pd.to_datetime(df4_test['release_date'], format="%Y-%m-%d",errors='coerce')
df4_test['id'] = pd.to_numeric(df4_test['id'])

Clean the price column such that Free = 0, and remove all outliers which are string

In [ ]:
unique_price = df4_test['price'].unique().tolist()
unique_price

[4.99,
 'Free To Play',
 'Free to Play',
 0.99,
 2.99,
 3.99,
 9.99,
 18.99,
 29.99,
 nan,
 'Free',
 10.99,
 1.59,
 14.99,
 1.99,
 59.99,
 8.99,
 6.99,
 7.99,
 39.99,
 19.99,
 7.49,
 12.99,
 5.99,
 2.49,
 15.99,
 1.25,
 24.99,
 17.99,
 61.99,
 3.49,
 11.99,
 13.99,
 'Free Demo',
 'Play for Free!',
 34.99,
 74.76,
 1.49,
 32.99,
 99.99,
 14.95,
 69.99,
 16.99,
 79.99,
 49.99,
 5.0,
 44.99,
 13.98,
 109.99,
 149.99,
 771.71,
 'Install Now',
 21.99,
 89.99,
 'Play WARMACHINE: Tactics Demo',
 0.98,
 139.92,
 4.29,
 64.99,
 'Free Mod',
 54.99,
 74.99,
 'Install Theme',
 0.89,
 'Third-party',
 0.5,
 'Play Now',
 299.99,
 1.29,
 119.99,
 3.0,
 15.0,
 5.49,
 23.99,
 20.99,
 1.39,
 'Free HITMAN™ Holiday Pack',
 36.99,
 4.49,
 2.0,
 4.0,
 9.0,
 234.99,
 1.95,
 1.5,
 6.66,
 27.99,
 10.49,
 26.99,
 399.99,
 31.99,
 20.0,
 40.0,
 3.33,
 129.99,
 199.99,
 22.99,
 38.85,
 71.7,
 59.95,
 995.0,
 27.49,
 3.39,
 6.0,
 19.95,
 499.99,
 16.06,
 4.68,
 131.4,
 44.98,
 202.76,
 1.0,
 2.3,
 0.95,
 172.24,
 2

In [14]:
non_float_price = []

In [15]:
for price in unique_price:
    if not isinstance(price, float):
        non_float_price.append(price)
        

In [16]:
non_float_price

['Free To Play',
 'Free to Play',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Free Movie']

In [17]:
for p in non_float_price:
    print(f'{p}: {(df4_test["price"] == p).sum()}')

Free To Play: 459
Free to Play: 517
Free: 868
Free Demo: 3
Play for Free!: 2
Install Now: 1
Play WARMACHINE: Tactics Demo: 1
Free Mod: 4
Install Theme: 1
Third-party: 2
Play Now: 2
Free HITMAN™ Holiday Pack: 1
Play the Demo: 1
Free Movie: 1


In [18]:
row_to_drop = ['Install Now', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Third-party', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free Movie', 'Play Now']

In [19]:
df4_test = df4_test[~df4_test['price'].isin(row_to_drop)]

In [20]:
df4_test['price'] = df4_test['price'].apply(lambda x: 0 if 'free' in str(x).lower() else x)

In [21]:
df4_test['price'] = pd.to_numeric(df4_test['price'])

In [22]:
df4_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31301 entries, 0 to 31310
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publisher     23696 non-null  object        
 1   genres        28534 non-null  object        
 2   title         31301 non-null  string        
 3   url           31301 non-null  object        
 4   release_date  28951 non-null  datetime64[ns]
 5   tags          31301 non-null  object        
 6   reviews_url   31300 non-null  object        
 7   specs         31301 non-null  object        
 8   price         29967 non-null  float64       
 9   early_access  31301 non-null  bool          
 10  id            31300 non-null  float64       
 11  developer     28408 non-null  object        
 12  sentiment     24363 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(2), object(8), string(1)
memory usage: 3.1+ MB


In [23]:
df4_test

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140.0,Kotoshiro,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980.0,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290.0,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400.0,彼岸领域,NaN
4,NaN,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaT,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31306,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640.0,"Nikita ""Ghost_RUS""",NaN
31307,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530.0,Sacada,NaN
31308,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660.0,Laush Dmitriy Sergeevich,NaN
31309,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870.0,"xropi,stev3ns",1 user reviews


Handle and clean the items column

In [24]:
def sanitize(x):
    if isinstance(x, list):
        # strip spaces and convert all to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        # check if director exists
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ""

In [25]:
from ast import literal_eval

def remove_quote(x):
    if isinstance(x, str):
        x = literal_eval(x)
        return '[' + ', '.join(x) + ']'

    return []

In [26]:
df4.fillna({'genres': '[]'}, inplace=True)

In [31]:
df4_test['publisher'] = df4_test['publisher'].apply(lambda x: [x.lower()] if isinstance(x, str) else [])
df4_test['developer'] = df4_test['developer'].apply(lambda x: [x.lower()] if isinstance(x, str) else [])

for feature in ['tags', 'genres', 'specs']:
    # df4_test[feature] = df4_test[feature].apply(remove_quote)
    # df4_test[feature] = df4_test[feature].apply(literal_eval)
    df4_test[feature] = df4_test[feature].apply(sanitize)
    # df4_test[feature] = df4_test[feature].apply(literal_eval)

In [32]:
df4_test

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,sentiment
0,[kotoshiro],"[action, casual, indie, simulation, strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[strategy, action, indie, casual, simulation]",http://steamcommunity.com/app/761140/reviews/?...,[single-player],4.99,False,761140.0,[kotoshiro],NaN
1,"[making fun, inc.]","[freetoplay, indie, rpg, strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[freetoplay, strategy, indie, rpg, cardgame, t...",http://steamcommunity.com/app/643980/reviews/?...,"[single-player, multi-player, onlinemulti-play...",0.00,False,643980.0,[secret level srl],Mostly Positive
2,[poolians.com],"[casual, freetoplay, indie, simulation, sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[freetoplay, simulation, sports, casual, indie...",http://steamcommunity.com/app/670290/reviews/?...,"[single-player, multi-player, onlinemulti-play...",0.00,False,670290.0,[poolians.com],Mostly Positive
3,[彼岸领域],"[action, adventure, casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[action, adventure, casual]",http://steamcommunity.com/app/767400/reviews/?...,[single-player],0.99,False,767400.0,[彼岸领域],NaN
4,[],,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaT,"[action, indie, casual, sports]",http://steamcommunity.com/app/773570/reviews/?...,"[single-player, fullcontrollersupport, htcvive...",2.99,False,773570.0,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31306,[ghost_rus games],"[casual, indie, simulation, strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[strategy, indie, casual, simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[single-player, steamachievements]",1.99,False,773640.0,"[nikita ""ghost_rus""]",NaN
31307,[sacada],"[casual, indie, strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[strategy, indie, casual]",http://steamcommunity.com/app/733530/reviews/?...,"[single-player, steamachievements, steamcloud,...",4.99,False,733530.0,[sacada],NaN
31308,[laush studio],"[indie, racing, simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[indie, simulation, racing]",http://steamcommunity.com/app/610660/reviews/?...,"[single-player, steamachievements, steamtradin...",1.99,False,610660.0,[laush dmitriy sergeevich],NaN
31309,[sixnails],"[casual, indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[indie, casual, puzzle, singleplayer, atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[single-player, steamachievements, steamcloud]",4.99,False,658870.0,"[xropi,stev3ns]",1 user reviews


In [37]:
df4_test['tags']

0            [strategy, action, indie, casual, simulation]
1        [freetoplay, strategy, indie, rpg, cardgame, t...
2        [freetoplay, simulation, sports, casual, indie...
3                              [action, adventure, casual]
4                          [action, indie, casual, sports]
                               ...                        
31306                [strategy, indie, casual, simulation]
31307                            [strategy, indie, casual]
31308                          [indie, simulation, racing]
31309    [indie, casual, puzzle, singleplayer, atmosphe...
31310    [earlyaccess, adventure, indie, action, simula...
Name: tags, Length: 31301, dtype: object

In [33]:
df4_test.to_csv('clean_df4.csv')